In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import os
from matplotlib.colors import LinearSegmentedColormap
import shapefile as shp
import collections
import time

In [2]:
# loading LUH2 dataset (transient and state) and calculating spatial index(x,y)
# state
LUH2_ssp = xr.open_dataset('/data1/home/whuang/i_python/LUH2/data/LUH2_ssp119_2015_2100.nc')
# transient
LUH2_trans = xr.open_dataset('/data1/home/whuang/i_python/LUH2/data/LUH2_trans_ssp119_2015_2100.nc')
# loading small girds information (500m x 500m) from classified maps
SPOT19 = xr.open_dataset('/work/ychen/ycmeet/SPOT_Classification/finalmap/TWD_1997_TM_Taiwan/7_class/2019_500m_7PFT.nc',decode_times=False)

# LUH2 dataset's resolution is 25km x 25km, and the classified maps are 500m x 500m, 
# so here we calculated spatial index(x,y) to know every large grid involved which small grids. 
Large_lon = LUH2_trans.lon.values
Large_lat = LUH2_trans.lat.values
Small_lon = SPOT19.nav_lon.values
Small_lat = SPOT19.nav_lat.values


# Select grid's index
lx = Large_lon.shape[0]
ly = Large_lon.shape[1]

XY_list = []
for x in range(lx):
    Y_list = []
    for y in range(ly):

        index = np.array(np.where((Small_lon<=Large_lon[x,y]+0.125) &
                                  (Small_lon>Large_lon[x,y]-0.125) &
                                  (Small_lat<=Large_lat[x,y]+0.125) &
                                  (Small_lat>Large_lat[x,y]-0.125) &
                                  (SPOT19.LU_TYPE[0].values==SPOT19.LU_TYPE[0].values))).T

        Y_list.append(index)
    XY_list.append(Y_list)



In [4]:
# processing data (to remove nan data)
for i in range(LUH2_trans.F2F[0].shape[0]):
    for j in range(LUH2_trans.F2F[0].shape[1]):
        if LUH2_trans.F2F[0,i,j]==LUH2_trans.F2F[0,i,j]: #1
            pass
        else:
            LUH2_trans.F2F[0,i,j]=0
        if LUH2_trans.A2F[0,i,j]==LUH2_trans.A2F[0,i,j]: #2
            pass
        else:
            LUH2_trans.A2F[0,i,j]=0            
        if LUH2_trans.G2F[0,i,j]==LUH2_trans.G2F[0,i,j]: #3
            pass
        else:
            LUH2_trans.G2F[0,i,j]=0            
        if LUH2_trans.B2F[0,i,j]==LUH2_trans.B2F[0,i,j]: #4
            pass
        else:
            LUH2_trans.B2F[0,i,j]=0            
        if LUH2_trans.F2A[0,i,j]==LUH2_trans.F2A[0,i,j]: #5
            pass
        else:
            LUH2_trans.F2A[0,i,j]=0
        if LUH2_trans.A2A[0,i,j]==LUH2_trans.A2A[0,i,j]: #6
            pass
        else:
            LUH2_trans.A2A[0,i,j]=0
        if LUH2_trans.G2A[0,i,j]==LUH2_trans.G2A[0,i,j]: #7
            pass
        else:
            LUH2_trans.G2A[0,i,j]=0
        
        if LUH2_trans.B2A[0,i,j]==LUH2_trans.B2A[0,i,j]: #8
            pass
        else:
            LUH2_trans.B2A[0,i,j]=0
        if LUH2_trans.F2G[0,i,j]==LUH2_trans.F2G[0,i,j]: #9
            pass
        else:
            LUH2_trans.F2G[0,i,j]=0
        if LUH2_trans.A2G[0,i,j]==LUH2_trans.A2G[0,i,j]: #10
            pass
        else:
            LUH2_trans.A2G[0,i,j]=0
        if LUH2_trans.G2G[0,i,j]==LUH2_trans.G2G[0,i,j]: #11
            pass
        else:
            LUH2_trans.G2G[0,i,j]=0
        if LUH2_trans.B2G[0,i,j]==LUH2_trans.B2G[0,i,j]: #12
            pass
        else:
            LUH2_trans.B2G[0,i,j]=0
        if LUH2_trans.F2B[0,i,j]==LUH2_trans.F2B[0,i,j]: #13
            pass
        else:
            LUH2_trans.F2B[0,i,j]=0
        if LUH2_trans.A2B[0,i,j]==LUH2_trans.A2B[0,i,j]: #14
            pass
        else:
            LUH2_trans.A2B[0,i,j]=0
        if LUH2_trans.G2B[0,i,j]==LUH2_trans.G2B[0,i,j]: #15
            pass
        else:
            LUH2_trans.G2B[0,i,j]=0
        if LUH2_trans.B2B[0,i,j]==LUH2_trans.B2B[0,i,j]: #16
            pass
        else:
            LUH2_trans.B2B[0,i,j]=0


In [5]:
# Merging transient data from three types of land-cover in 2015
# Agi.+Grass = AG
# F2AG, F2B, AG2F, AG2B, B2F, B2AG

TS_LC_2015 = np.zeros((6,15,8))
TS_LC_2015[0][:] = LUH2_trans.F2A.values[0][:]+LUH2_trans.F2G.values[0][:]
TS_LC_2015[1][:] = LUH2_trans.F2B.values[0][:]
TS_LC_2015[2][:] = LUH2_trans.A2F.values[0][:]+LUH2_trans.G2F.values[0][:]
TS_LC_2015[3][:] = LUH2_trans.A2B.values[0][:]+LUH2_trans.G2B.values[0][:]
TS_LC_2015[4][:] = LUH2_trans.B2F.values[0][:]
TS_LC_2015[5][:] = LUH2_trans.B2A.values[0][:]+LUH2_trans.B2G.values[0][:]

In [6]:
## Definition of functions
def LU_Processing(map):
    LU = np.zeros((map.shape))
    LU[:] = map[:]
    for i in range(LU.shape[0]):
        for j in range(LU.shape[1]):
            if LU[i,j]!=LU[i,j]:
                LU[i,j]=-999
    unique, counts = np.unique(LU, return_counts=True)
    return LU,unique, counts


def Count_4type_map(Array):

    LU = np.zeros((Array.shape))
    LU[:] = Array[:]
    
    for i in range(LU.shape[0]):
        for j in range(LU.shape[1]):
            if LU[i,j]!=LU[i,j]:
                LU[i,j]=-999
    unique, counts = np.unique(LU, return_counts=True)
    LC_list = [0,0,0,0]
    
    for i in range(len(unique)):
        if unique[i]==1:
            LC_list[0]=counts[i]
        elif unique[i]==2:
            LC_list[1]=counts[i]
        elif unique[i]==4:
            LC_list[2]=counts[i]
        elif unique[i]==8:
            LC_list[3]=counts[i]  
        else:
            pass
    return LC_list

In [7]:
# Method 2
# loading 2019 SOPT image and 2015 Land-use map
SPOT19 = xr.open_dataset('/work/ychen/ycmeet/SPOT_Classification/finalmap/TWD_1997_TM_Taiwan/7_class/2019_500m_7PFT.nc',decode_times=False)
LU_2015 = np.load("/data1/home/whuang/i_python/BMF/data/Four_Tpyes_Maps/LU4T_2015.npy")
# Shape definition of output array
New_Array = np.zeros((4,15,8))
New_Array[:] = np.nan

# transient data of SSP119 in 2029[15]
LUH2_ssp_n = [LUH2_ssp.Forest[15], LUH2_ssp.Grassland[15]+LUH2_ssp.Agriculture[15], LUH2_ssp.Builtup[15], LUH2_ssp.Water[15]]

lx = Large_lon.shape[0]
ly = Large_lon.shape[1]
for x in range(lx):
    for y in range(ly):
       
        # 2015 grids
        index = XY_list[x][y]
        
        if len(index)>0:

            filter_array = np.zeros((900, 747))
            filter_array[:] = np.nan

            for i ,j in index:
                filter_array[i,j]=1
            
            TYPE4=[]
            for n in range(4): #4 LU tpyes

                Type_count=Count_4type_map((LU_2015)*filter_array)[n]

                # Allocate the addition of gross exchange area
                # LUH2:25km*25km
                # number of grids: (25km*25km)/(0.5km*0.5km)=2500(incorrect)
                # Grids areas are different
                
                if LUH2_ssp_n[n][x,y]==LUH2_ssp_n[n][x,y]:
                    New_Array[n,x,y] = ((int(LUH2_ssp_n[n][x,y]*len(index))-Type_count))
                else:
                    New_Array[n,x,y] = 0
            else:
                pass
            
            


In [9]:
# A linear function of LCC from 2019 to 2029 (total 10 yrs)
# Use unit step function to add d(biomass) from the first 10 years
def nuit_step_p(t):
    if t>=0:
        z=0
    else:
        z=-1
    return z

def dBiomass(t):
    
    dF = np.heaviside(nuit_step_p(10-t), int(F/10))
    dAG = np.heaviside(nuit_step_p(10-t), int(AG/10))
    dB = np.heaviside(nuit_step_p(10-t), int(B/10))
    
    return dF, dAG, dB

F=0
AG=0
B=0
for x in range(LUH2_ssp.lon.shape[0]):
    for y in range(LUH2_ssp.lon.shape[1]):
        #print(New_Array[:,x,y])
        if New_Array[0,x,y]==New_Array[0,x,y]:
            F = New_Array[0,x,y]+F
            AG = New_Array[1,x,y]+AG
            B = New_Array[2,x,y]+B
print(F,AG,B)

-23631.0 16165.0 8206.0


In [10]:
TLUSS = (np.load("/data1/home/whuang/i_python/BMF/data/Four_Tpyes_Maps/Potential_change/LU4T_1904.npy")*52+
        np.load("/data1/home/whuang/i_python/BMF/data/Four_Tpyes_Maps/Potential_change/LU4T_1956.npy")*26+
        np.load("/data1/home/whuang/i_python/BMF/data/Four_Tpyes_Maps/Potential_change/LU4T_1982.npy")*13+
        np.load("/data1/home/whuang/i_python/BMF/data/Four_Tpyes_Maps/Potential_change/LU4T_1995.npy")*5+
        np.load("/data1/home/whuang/i_python/BMF/data/Four_Tpyes_Maps/Potential_change/LU4T_2000.npy")*10+
        np.load("/data1/home/whuang/i_python/BMF/data/Four_Tpyes_Maps/Potential_change/LU4T_2010.npy")*5+
        np.load("/data1/home/whuang/i_python/BMF/data/Four_Tpyes_Maps/Potential_change/LU4T_2015.npy"))
TLUSS_Percentage = (TLUSS/112)
np.save('/data1/home/whuang/i_python/BMF/data/SaveArray/temp_save', TLUSS_Percentage)


# Spatial strategy (No years weight)
def LUSS4T(map):
    LU = map
    array = np.zeros((4,900, 747))
    LU_code= [1, 2, 8, 13]

    for i in range(LU.shape[0]):
        for j in range(LU.shape[1]):
            for k in range(len(LU_code)):
                if LU[i,j] == LU_code[k]:
                    array[k,i,j]=array[k,i,j]+1
                    try:
                        array[k,i+1,j]=array[k,i+1,j]+1
                    except IndexError:
                        pass
                    try:
                        array[k,i,j+1]=array[k,i,j+1]+1
                    except IndexError:
                        pass
                    try:
                        array[k,i+1,j+1]=array[k,i+1,j+1]+1
                    except IndexError:
                        pass
                    try:
                        array[k,i-1,j]=array[k,i-1,j]+1
                    except IndexError:
                        pass
                    try:
                        array[k,i,j-1]=array[k,i,j-1]+1
                    except IndexError:
                        pass
                    try:
                        array[k,i-1,j-1]=array[k,i-1,j-1]+1
                    except IndexError:
                        pass
                    try:
                        array[k,i+1,j-1]=array[k,i+1,j-1]+1
                    except IndexError:
                        pass
                    try:
                        array[k,i-1,j+1]=array[k,i-1,j+1]+1
                    except IndexError:
                        pass
    for i in range(4):
        array[i] = array[i]*block_mesh
        
    PLUSS = (array/9*100)*block_mesh
    return PLUSS

In [ ]:
#       1      4      2     8     42
#     Forest Agri  Grass  City   Agri(I)
#
# F     0     -3     -1    -7    -41
# A     3      0      2    -4    -38
# G     1     -2      0    -6    -40
# C     7      4      6     0    -36
# I    41     38     40    36     0

# Make a Urban Map to count 60 years
#Urban_Count = np.zeros((900, 747))
#Urban_Count=np.load('/data1/home/whuang/i_python/BMF/data/Four_Tpyes_Maps/LU4T_2015.npy')
#Urban_Count[np.where(Urban_Count!=8)]=0
#Urban_Count[np.where(Urban_Count==8)]=1
#

Year_List = [i for i in range(2015,2100)]

Check_Tr = []

for yy in range(1,len(Year_List)):

    print("Year:",Year_List[yy])
    
    
    # 0:F2F, 1:F2GA, 2:F2B, 3:GA2F, 4:GA2GA, 5:GA2B, 6:B2F, 7:B2GA, 8:B2B
    TS_Data_One_Array = np.zeros((9,15,8))
    TS_Data_One_Array[:] = np.nan
    TS_Data_One_Array[0] = LUH2_trans.F2F.values[yy][:]
    TS_Data_One_Array[1] = LUH2_trans.F2A.values[yy][:]+LUH2_trans.F2G.values[yy][:]
    TS_Data_One_Array[2] = LUH2_trans.F2B.values[yy][:]
    TS_Data_One_Array[3] = LUH2_trans.A2F.values[yy][:]+LUH2_trans.G2F.values[yy][:]
    TS_Data_One_Array[4] = LUH2_trans.G2G.values[yy][:]+LUH2_trans.A2A.values[yy][:]
    TS_Data_One_Array[5] = LUH2_trans.A2B.values[yy][:]+LUH2_trans.G2B.values[yy][:]
    TS_Data_One_Array[6] = LUH2_trans.B2F.values[yy][:]
    TS_Data_One_Array[7] = LUH2_trans.B2A.values[yy][:]+LUH2_trans.B2G.values[yy][:]
    TS_Data_One_Array[8] = LUH2_trans.B2B.values[yy][:]
    #
    
    # number of LC girds
    start = time.time()
    Temp_Array = np.zeros((9,15,8))
    Temp_Array[:] = np.nan
    
    index_list = []
    for x in range(lx):
        for y in range(ly):

            index = XY_list[x][y]
            index_list.append(XY_list[x][y])
            if len(index)>0:
                for n in range(9): #4 LU tpyes
                
                    if TS_Data_One_Array[n][x,y]==TS_Data_One_Array[n][x,y]:
                        Temp_Array[n,x,y] = int(TS_Data_One_Array[n][x,y]*len(index))
                    else:
                        Temp_Array[n,x,y] = 0
                else:
                    pass
                
    
    end = time.time()
    print("執行時間3：%f 秒" % (end - start))
    
    F2F = np.nansum(Temp_Array[0])
    F2GA = np.nansum(Temp_Array[1])+dBiomass(yy)[1]
    F2B = np.nansum(Temp_Array[2])+dBiomass(yy)[2]

    GA2F = np.nansum(Temp_Array[3])
    GA2GA = np.nansum(Temp_Array[4])
    GA2B = np.nansum(Temp_Array[5])

    B2F = np.nansum(Temp_Array[6])
    B2GA = np.nansum(Temp_Array[7])
    B2B = np.nansum(Temp_Array[8])
    
    F2F=GA2GA=B2B=0
    
    #print(F2F, F2GA, F2B, GA2F, GA2GA, GA2B, B2F, B2GA, B2B)
    
    
    TGLC_Type_LUH2 = np.array([F2F, F2GA, F2B,
                               GA2F, GA2GA, GA2B,
                               B2F, B2GA, B2B])
    # For track
    Check_Tr.append(TGLC_Type_LUH2)
    if yy==1:
        Year_Array = np.load('/data1/home/whuang/i_python/BMF/data/Four_Tpyes_Maps/LU4T_2015.npy')
        
    else:
        Year_Array = np.load('/data1/home/whuang/i_python/BMF/data/SaveArray/SSP245/LUH2_245_{:}.npy'.format(Year_List[yy-1])) # LU_2015
  

    # Step 3 
    # Ranking elements of array 
    def max_ranking_array(MaxPercentageA, index):
        unique, counts = np.unique(MaxPercentageA, return_counts=True)
        TMK=[]
        for i in range(len(unique)-1,0,-1):  
            TMK.append(np.array(np.where(MaxPercentageA==unique[i])).T)
        TMK = np.vstack(TMK)
        return TMK



    # Step 5
    CGEA = TGLC_Type_LUH2
    
    # Step 6
    F1=0
    F2=0
    F3=0
    GA1=0
    GA2=0
    GA3=0
    B1=0
    B2=0
    B3=0

    NN=1
    
    TLUSS_Percentage = np.load('/data1/home/whuang/i_python/BMF/data/SaveArray/temp_save.npy')
    
    # Assign the land-use grids in an new map(final_array)
    final_array = np.zeros((900, 747))
    final_array[:] = Year_Array
    #
    bool_array = np.zeros((900, 747))
    #
    while (F1<CGEA[0] or F2<CGEA[1] or F3<CGEA[2] or 
           GA1<CGEA[3] or GA2<CGEA[4] or GA3<CGEA[5] or 
           B1<CGEA[6] or B2<CGEA[7] or B3<CGEA[8]):
        
        start = time.time()
        print("NN:",NN)

        if NN==4:
            break
        # Find the Maximum Ranking nubmer
        TEST_A = np.zeros((900, 747))
        for i in range(900):
            for j in range(747):
                flat=TLUSS_Percentage[:,i,j].flatten()
                flat.sort()
                # -2 is the second large number
                # -1 is the maximum number
                TEST_A[i,j] = flat[-NN] 
        index = np.vstack(index_list)
        BR = max_ranking_array(TEST_A,index)    

        
        if len(BR)==0:
            break
        for i ,j in BR:

            BOOL = (TLUSS_Percentage[:, i ,j]==(TEST_A)[i ,j])
            if (CGEA == [F1,F2,F3,GA1,GA2,GA3,B1,B2,B3]).all():
                pass
            elif bool_array[i ,j]>0:
                pass
            else:
                if Year_Array[i ,j]==1:
                    if BOOL[0]:
                        if F1==CGEA[0]:
                            pass
                        else:
                            final_array[i ,j]=1
                            F1+=1
                            bool_array[i ,j]+=1
                    elif BOOL[1]:
                        if F2==CGEA[1]:
                            pass
                        else:
                            final_array[i ,j]=2
                            F2+=1
                            bool_array[i ,j]+=1
                    elif BOOL[2]:
                        if F3==CGEA[2]:
                            pass
                        else:
                            final_array[i ,j]=8
                            F3+=1
                            bool_array[i ,j]+=1
                            #Urban_Count[i ,j]=1     

                if Year_Array[i ,j]==2:
                    if BOOL[0]:
                        if GA1==CGEA[3]:
                            pass
                        else:
                            final_array[i ,j]=1
                            GA1+=1
                            bool_array[i ,j]+=1
                    elif BOOL[1]:
                        if GA2==CGEA[4]:
                            pass
                        else:
                            final_array[i ,j]=2
                            GA2+=1
                            bool_array[i ,j]+=1
                    elif BOOL[2]:
                        if GA3==CGEA[5]:
                            pass
                        else:
                            final_array[i ,j]=8
                            GA3+=1
                            bool_array[i ,j]+=1
                            #Urban_Count[i ,j]=1

                if Year_Array[i ,j]==8:
                    #if Urban_Count[i ,j]>60:
                    if BOOL[0]:
                        if B1==CGEA[6]:
                            pass
                        else:
                            final_array[i ,j]=1
                            B1+=1
                            bool_array[i ,j]+=1
                    elif BOOL[1]:
                        if B2==CGEA[7]:
                            pass
                        else:
                            final_array[i ,j]=2
                            B2+=1
                            bool_array[i ,j]+=1
                    elif BOOL[2]:
                        if B3==CGEA[8]:
                            pass
                        else:
                            final_array[i ,j]=8
                            B3+=1
                            bool_array[i ,j]+=1
                            #Urban_Count[i ,j]=1
                    #else:
                    #    pass

        NN+=1  
        end = time.time()
        print("執行時間4：%f 秒" % (end - start))
    
    #for i in range(Urban_Count.shape[0]):
    #    for j in range(Urban_Count.shape[1]):
    #        if Urban_Count[i ,j]>0:
    #            Urban_Count[i ,j]+=1
     
    np.save('/data1/home/whuang/i_python/BMF/data/SaveArray/SSP245/LUH2_245_{:}'.format(Year_List[yy]), final_array)
    temp_save = (TLUSS_Percentage*(112+yy)+ Percetage_LUSS(LUSS4T(final_array)))/(113+yy)
    np.save('/data1/home/whuang/i_python/BMF/data/SaveArray/temp_save', temp_save)
    
    # Release memory
    del temp_save
    del TLUSS_Percentage
    del TGLC_Type_LUH2
    # Release memory
    print("")
    gc.collect()